## 1. 임베딩 & Qdrant 업서트

In [8]:
import os
import uuid
import json
import torch
from pathlib import Path
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels

# ===== 모델 매핑 =====
MODEL_NAME = "bge-ko"   # ← 여기만 바꿔주면 됨 ("e5", "koe5", "bge-ko", "bge-m3", "kure")

MODEL_MAP = {
    "e5": "intfloat/multilingual-e5-small",
    "e5-base": "intfloat/multilingual-e5-base",
    "koe5": "intfloat/KoE5-large",
    "bge-ko": "dragonkue/bge-m3-ko",
    "bge-m3": "BAAI/bge-m3",
    "kure": "nlpai-lab/KURE-v1",
}

EMBED_MODEL_NAME = MODEL_MAP[MODEL_NAME]
COLLECTION       = "audit_chunks"
BATCH            = 64
QDRANT_PATH = f"/Users/dan/Desktop/snu_project/git_제출용/data/vector_store/final-sjchunk/{MODEL_NAME}-qdrant_db"
CHUNK_FILE = Path("/Users/dan/Desktop/snu_project/git_제출용/data/processed/enhanced_vector_chunks_9_24.jsonl")

# 토크나이저 포크 경고 끄기 (권장)
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# ===== 디바이스 선택 (MPS > CUDA > CPU) =====
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"[INFO] Using device: {device}")

# ===== 모델 로드 =====
model = SentenceTransformer(EMBED_MODEL_NAME, device=device)
dim = model.get_sentence_embedding_dimension()
print(f"[INFO] Loaded model: {MODEL_NAME} ({EMBED_MODEL_NAME}), dim={dim}")

# ===== corpus 로드 =====
assert CHUNK_FILE.exists(), f"청킹 파일을 찾을 수 없습니다: {CHUNK_FILE}"

corpus = []
with open(CHUNK_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        corpus.append(json.loads(line))

print(f"[INFO] Loaded corpus: {len(corpus)} chunks")

client = None
try:
    # ===== Qdrant 연결 (임베디드 모드) =====
    client = QdrantClient(path=QDRANT_PATH)

    # 컬렉션 생성 (존재하지 않을 때만)
    if not client.collection_exists(COLLECTION):
        client.create_collection(
            collection_name=COLLECTION,
            vectors_config=qmodels.VectorParams(
                size=dim,
                distance=qmodels.Distance.COSINE
            ),
            optimizers_config=qmodels.OptimizersConfigDiff(indexing_threshold=20000),
            hnsw_config=qmodels.HnswConfigDiff(m=32, ef_construct=256),
        )
        print(f"[INFO] Created collection: {COLLECTION}")
    else:
        print(f"[INFO] Using existing collection: {COLLECTION}")

    # ===== 업서트 =====
    pending_points = []
    for i in tqdm(range(0, len(corpus), BATCH), desc=f"Upserting ({MODEL_NAME})"):
        batch = corpus[i:i+BATCH]
        texts = [x["text"] for x in batch]

        vecs = model.encode(
            texts,
            normalize_embeddings=True,
            convert_to_numpy=True,
            show_progress_bar=False,
            batch_size=BATCH,
        ).astype("float32")

        pending_points.clear()
        for x, v in zip(batch, vecs):
            pid = str(uuid.uuid4())   # ✅ 무조건 올바른 UUID 생성
            payload = {**x.get("metadata", {}), "text": x["text"]}
            pending_points.append(
                qmodels.PointStruct(id=pid, vector=v.tolist(), payload=payload)
            )


        client.upsert(collection_name=COLLECTION, points=pending_points, wait=False)

    try:
        client.update_collection(
            collection_name=COLLECTION,
            hnsw_config=qmodels.HnswConfigDiff(ef_construct=256),
            optimizers_config=qmodels.OptimizersConfigDiff(default_segment_number=4),
        )
    except Exception:
        pass

    print(f"[INFO] Upsert done: {len(corpus)} → DB path={QDRANT_PATH}")

finally:
    if client is not None:
        try:
            client.close()
        except Exception:
            pass

[INFO] Using device: mps
[INFO] Loaded model: bge-ko (dragonkue/bge-m3-ko), dim=1024
[INFO] Loaded corpus: 2292 chunks
[INFO] Using existing collection: audit_chunks


Upserting (bge-ko): 100%|██████████| 36/36 [00:18<00:00,  1.92it/s]

[INFO] Upsert done: 2292 → DB path=/Users/dan/Desktop/snu_project/git_제출용/data/vector_store/final-sjchunk/bge-ko-qdrant_db


## 2. Retriever 성능 테스트

In [2]:
import numpy as np
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
import re
from qdrant_client.http import models as qmodels


# ===== 임베딩 모델 로드 =====
embed_model = SentenceTransformer("dragonkue/bge-m3-ko")

# ===== 숫자만 추출 함수 =====
def extract_numbers(text: str) -> str:
    if text is None:
        return ""
    return "".join(re.findall(r"\d+", text.replace(",", "")))

# ===== Retriever 함수 =====
def extract_year_from_query(query: str):
    """질문에서 연도(4자리 숫자) 추출"""
    match = re.search(r"(20\d{2}|19\d{2})", query)
    if match:
        return int(match.group(1))
    return None
    
def dense_search(query: str, model, client, collection_name: str,
                 top_k: int = 3, ground_truth=None):
    """
    Qdrant query_points 기반 Dense Retriever 함수 + 성능지표 계산
    """
    # 0. 질문에서 연도 추출
    year = extract_year_from_query(query)
    
    # 1. 쿼리 임베딩 생성
    qv = model.encode(query, normalize_embeddings=True).tolist()

    # 2. Qdrant 검색 실행 (연도 필터 있으면 적용)
    query_filter = None
    if year:
        query_filter = qmodels.Filter(
            must=[
                qmodels.FieldCondition(
                    key="report_year",
                    match=qmodels.MatchValue(value=year)
                )
            ]
        )

    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=query_filter  # ✅ 필터 적용
    )

    # 3. 결과 정리 (payload 전체 반영)
    output = []
    for r in results.points:
        payload = r.payload or {}
        result_item = {
            "score": r.score,
            "text": payload.get("text")
        }
        # metadata 안의 모든 키-값을 추가
        if "metadata" in payload:
            result_item.update(payload["metadata"])
        else:
            # 혹시 metadata 키 없이 flat하게 들어온 경우
            result_item.update(payload)
        output.append(result_item)

    # 4. 성능 지표 계산
    metrics = {}
    if ground_truth:
        # 정답 숫자만 추출
        normalized_gt = [extract_numbers(gt) for gt in ground_truth]

        # 각 검색 결과가 정답과 매칭되는지 여부
        used = set()
        relevances = []
        for r in output:
            nums = extract_numbers(r["text"])
            hit = 0
            for gt in normalized_gt:
                if gt and gt in nums and gt not in used:
                    hit = 1
                    used.add(gt)
                    break
            relevances.append(hit)

        # Precision@3
        precision = sum(relevances) / top_k if top_k > 0 else 0.0
        metrics["Precision@3"] = precision

        # Recall@3 (전체 정답 대비 비율)
        recall = min(sum(relevances), len(normalized_gt)) / len(normalized_gt) if normalized_gt else 0.0
        metrics["Recall@3"] = recall

        # F1@3
        if precision + recall > 0:
            metrics["F1@3"] = 2 * (precision * recall) / (precision + recall)
        else:
            metrics["F1@3"] = 0.0

        # MRR
        rr = 0.0
        for rank, rel in enumerate(relevances, 1):
            if rel == 1:
                rr = 1.0 / rank
                break
        metrics["MRR"] = rr

        # nDCG@k
        dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevances))
        ideal_hits = min(len(normalized_gt), top_k)   # 최대 정답 수
        idcg = sum(1.0 / np.log2(idx + 2) for idx in range(ideal_hits))
        metrics["nDCG@3"] = dcg / idcg if idcg > 0 else 0.0

    return output, metrics


# =========================
# 실행 예시
# =========================
if __name__ == "__main__":
    QDRANT_PATH = "/Users/dan/Desktop/snu_project/git_제출용/data/vector_store/final-sjchunk/bge-ko-qdrant_db"
    client = QdrantClient(path=QDRANT_PATH)

    collection_name = "audit_chunks"

    questions = [
    "2014년 재무상태표 상 당기 유동자산은 얼마인가?",
    "2014년 현금흐름표 상 당기 영업활동 현금흐름은 얼마인가?",
    "2015년 당기 비유동자산은 재무상태표에서 얼마인가?",
    "2015년 손익계산서 상 당기순이익은 얼마인가?",
    "2016년 재무상태표 상 당기 단기금융상품은 얼마인가요?",
    "2016년 포괄손익계산서 상 당기 총포괄이익은 얼마니?",
    "2016년 자본변동표 상 자기주식의 취득은 얼마인가?",
    "2017년 당기 매출채권은 재무상태표에 따르면 얼마냐?",
    "2017년 재무상태표상 전기 현금및현금성자산은 얼마입니까?",
    "2018년 당기 미수금은 재무상태표에서 얼마인가?",
    "2018년 손익계산서상 매출총이익은 얼마인가요?",
    "2019년 재무상태표상 종속기업, 관계기업 및 공동기업 투자는 얼마인가요?",
    "2019년 현금흐름표 상 이익잉여금 배당은 얼마인가요?",
    "2019년 손익계산서상 기본주당이익은 얼마인가요?",
    "2020년 재무상태표 상 자산총계는?",
    "2020년 손익계산서 상 판매비와관리비는 얼마인가요?",
    "2021년 재무상태표상 당기 기타포괄손익-공정가치금융자산은 얼마인가요?",
    #"2021년 재무상태표에서 당기 유동비율을 계산하면 얼마인가요?",
    "2021년 손익계산서 상 당기 금융비용은 얼마인가요?",
    "2022년 재무상태표상 당기 비유동부채는 얼마인가?",
    "2022년 손익계산서 상 당기 법인세비용은 얼마니?",
    "2022년 당기 현금흐름표 상 투자활동 현금흐름은 얼마인가?",
    "2023년 재무상태표 상 재고자산은 얼마인가?",
    "2023년 손익계산서 상 당기 영업이익은 얼마인가?",
    "2024년에는 재무상태표상 당기 무형자산이 얼마야?",
    "2024년 재무상태표 상 당기 우선주자본금은 얼마인가?",
    "2024년 손익계산서상 당기 법인세비용은 얼마야?",
    "2017년 재무상태표상 당기 매각예정분류자산은 얼마인가요?",
    ]
    
    answers = [
        "62,054,773",
        "18,653,817",
        "101,967,575",
        "12,238,469",
        "30,170,656",
        "11,887,806",
        "(7,707,938)",
        "27,881,777",
        "3,778,371",
        "1,515,079",
        "68,715,364",
        "56,571,252",
        "(9,618,210)",
        "2,260",
        "229,664,427",
        "29,038,798",
        "1,662,532",
        #"1.38",
        "3,698,675",
        "4,581,512",
        "4,273,142",
        "(28,123,886)",
        "29,338,151",
        "(11,526,297)",
        "10,496,956",
        "119,467",
        "(1,832,987)",
        "-"
    ]

    all_metrics = []

    for q, a in zip(questions, answers):
        results, metrics = dense_search(
            query=q,
            model=embed_model,
            client=client,
            collection_name=collection_name,
            top_k=3,
            ground_truth=[a]
        )

        print("=" * 100)
        print(f"질문: {q}")

        # Top-3 결과 모두 출력
        if results:
            print("\n[검색 결과 Top-3]")
            for rank, r in enumerate(results, 1):
                print(f"{rank}위: {r['text']}")
                print(f"   출처: {r['report_year']}년, (score={r['score']:.4f})\n")
        else:
            print("검색 결과 없음")

        print("\n[정답]")
        print(a)

        print("\n[성능 지표]")
        print(
            f" Precision@3={metrics.get('Precision@3', 0):.2f},"
            f" Recall@3={metrics.get('Recall@3', 0):.2f},"
            f" F1@3={metrics.get('F1@3', 0):.2f},"
            f" nDCG@3={metrics.get('nDCG@3', 0):.2f},"
            f" MRR={metrics.get('MRR', 0):.2f}"
        )

        all_metrics.append(metrics)

    # 평균 성능 지표
    df = pd.DataFrame(all_metrics)
    print("\n" + "=" * 100)
    print("=== 평균 성능 지표 ===")
    print(df.mean().round(3))

    client.close()


질문: 2014년 재무상태표 상 당기 유동자산은 얼마인가?

[검색 결과 Top-3]
1위: 재무상태표에서 2014년 (당기) 유동자산는 62,054,773백만원입니다.
   출처: 2014년, (score=0.8690)

2위: 재무상태표에서 2014년 (당기) 유동자산는 62,054,773백만원입니다.
   출처: 2014년, (score=0.8690)

3위: 재무상태표에서 2014년 (당기) 유동자산는 62,054,773백만원입니다.
   출처: 2014년, (score=0.8690)


[정답]
62,054,773

[성능 지표]
 Precision@3=0.33, Recall@3=1.00, F1@3=0.50, nDCG@3=1.00, MRR=1.00
질문: 2014년 현금흐름표 상 당기 영업활동 현금흐름은 얼마인가?

[검색 결과 Top-3]
1위: 현금흐름표에서 2014년 (당기) 영업활동현금흐름는 18,653,817백만원입니다.
   출처: 2014년, (score=0.8249)

2위: 현금흐름표에서 2014년 (당기) 영업활동현금흐름는 18,653,817백만원입니다.
   출처: 2014년, (score=0.8249)

3위: 현금흐름표에서 2014년 (당기) 영업활동현금흐름는 18,653,817백만원입니다.
   출처: 2014년, (score=0.8249)


[정답]
18,653,817

[성능 지표]
 Precision@3=0.33, Recall@3=1.00, F1@3=0.50, nDCG@3=1.00, MRR=1.00
질문: 2015년 당기 비유동자산은 재무상태표에서 얼마인가?

[검색 결과 Top-3]
1위: 재무상태표에서 2015년 (당기) 비유동자산는 101,967,575백만원입니다.
   출처: 2015년, (score=0.8659)

2위: 재무상태표에서 2015년 (당기) 비유동자산는 101,967,575백만원입니다.
   출처: 2015년, (score=0.8659)

3위: 재무상태표에서 2015년 (당기) 비유동자산는 10

In [ ]:
import numpy as np
from pathlib import Path
# from llama_cpp import Llama  # 하단에서 import
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
import re
from qdrant_client.http import models as qmodels


# ===== 임베딩 모델 로드 =====
embed_model = SentenceTransformer("dragonkue/bge-m3-ko")

# ===== Qdrant 클라이언트 =====
QDRANT_PATH = "/Users/dan/Desktop/snu_project/git_제출용/data/vector_store/final-sjchunk/bge-ko-qdrant_db"
client = QdrantClient(path=QDRANT_PATH)

collection_name = "audit_chunks"

# ===== dense_search 함수 import =====
# (예빈씨가 이전에 정의한 dense_search 그대로 사용한다고 가정)
def extract_year_from_query(query: str):
    """질문에서 연도(4자리 숫자) 추출"""
    match = re.search(r"(20\d{2}|19\d{2})", query)
    if match:
        return int(match.group(1))
    return None

def extract_numbers(text: str) -> str:
    """텍스트에서 숫자만 추출"""
    if text is None:
        return ""
    return "".join(re.findall(r"\d+", text.replace(",", "")))
    
def dense_search(query: str, model, client, collection_name: str,
                 top_k: int = 50, ground_truth=None):
    """
    Qdrant query_points 기반 Dense Retriever 함수 + 성능지표 계산
    """
    # 0. 질문에서 연도 추출
    year = extract_year_from_query(query)
    
    # 1. 쿼리 임베딩 생성
    qv = model.encode(query, normalize_embeddings=True).tolist()

    # 2. Qdrant 검색 실행 (연도 필터 있으면 적용)
    query_filter = None
    if year:
        query_filter = qmodels.Filter(
            must=[
                qmodels.FieldCondition(
                    key="report_year",
                    match=qmodels.MatchValue(value=year)
                )
            ]
        )

    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=query_filter  # ✅ 필터 적용
    )

    # 3. 결과 정리 (payload 전체 반영)
    output = []
    for r in results.points:
        payload = r.payload or {}
        result_item = {
            "score": r.score,
            "text": payload.get("text")
        }
        # metadata 안의 모든 키-값을 추가
        if "metadata" in payload:
            result_item.update(payload["metadata"])
        else:
            # 혹시 metadata 키 없이 flat하게 들어온 경우
            result_item.update(payload)
        output.append(result_item)

    # 4. 성능 지표 계산
    metrics = {}
    if ground_truth:
        # 정답 숫자만 추출
        normalized_gt = [extract_numbers(gt) for gt in ground_truth]

        # 각 검색 결과가 정답과 매칭되는지 여부
        used = set()
        relevances = []
        for r in output:
            nums = extract_numbers(r["text"])
            hit = 0
            for gt in normalized_gt:
                if gt and gt in nums and gt not in used:
                    hit = 1
                    used.add(gt)
                    break
            relevances.append(hit)

        # Precision@3
        precision = sum(relevances) / top_k if top_k > 0 else 0.0
        metrics["Precision@3"] = precision

        # Recall@3 (전체 정답 대비 비율)
        recall = min(sum(relevances), len(normalized_gt)) / len(normalized_gt) if normalized_gt else 0.0
        metrics["Recall@3"] = recall

        # F1@3
        if precision + recall > 0:
            metrics["F1@3"] = 2 * (precision * recall) / (precision + recall)
        else:
            metrics["F1@3"] = 0.0

        # MRR
        rr = 0.0
        for rank, rel in enumerate(relevances, 1):
            if rel == 1:
                rr = 1.0 / rank
                break
        metrics["MRR"] = rr

        # nDCG@k
        dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevances))
        ideal_hits = min(len(normalized_gt), top_k)   # 최대 정답 수
        idcg = sum(1.0 / np.log2(idx + 2) for idx in range(ideal_hits))
        metrics["nDCG@3"] = dcg / idcg if idcg > 0 else 0.0

    return output, metrics
    
# ===== Zephyr 모델 로드 =====
model_path = Path("/Users/dan/Desktop/snu_project/models/zephyr-7b-beta.Q4_K_M.gguf").resolve()

# from llama_cpp import Llama  # Cell 6에서 로드됨
# llm = Llama(
#     model_path=str(model_path),
#     n_ctx=4096,
#     n_threads=8,
#     n_gpu_layers=35
# )

# ===== 간단한 검색 테스트 함수 =====
def simple_search_test(query: str, model, client, collection_name: str, top_k: int = 10):
    """LLM 없이 검색 결과만 확인하는 함수"""
    results, _ = dense_search(
        query=query,
        model=model,
        client=client,
        collection_name=collection_name,
        top_k=top_k
    )
    
    print(f"🔍 검색 결과 ({len(results)}개):")
    print("=" * 100)
    
    for i, r in enumerate(results, 1):
        print(f"\n{i}. 스코어: {r['score']:.4f}")
        print(f"   연도: {r.get('report_year', 'N/A')}")
        print(f"   텍스트: {r['text'][:200]}...")
        
        # metadata 정보 출력
        metadata_keys = ['account_id', 'account_name', 'parent_id', 'level', 'hierarchy', 
                        'is_total', 'is_subtotal', 'period_type', 'statement_type']
        metadata_info = []
        for key in metadata_keys:
            if key in r:
                metadata_info.append(f"{key}: {r[key]}")
        if metadata_info:
            print(f"   📊 메타데이터: {', '.join(metadata_info)}")
        print("-" * 80)

    return results

# ===== RAG Pipeline (LLM 없이 검색만) =====
def rag_pipeline_simple(query: str, model, client, collection_name: str, top_k: int = 10):
    """LLM 없이 검색 결과만 반환하는 간단한 RAG"""
    print(f"💬 질문: {query}")
    print()
    
    # 검색 수행
    results = simple_search_test(query, model, client, collection_name, top_k)
    
    # 간단한 요약 정보
    print(f"\n📋 요약:")
    print(f"   - 총 {len(results)}개의 관련 문서를 찾았습니다.")
    
    if results:
        years = list(set([r.get('report_year', 'N/A') for r in results if r.get('report_year')]))
        if years:
            print(f"   - 관련 연도: {', '.join(map(str, sorted(years)))}")
        
        # 계층 정보가 있는 경우
        hierarchies = [r.get('hierarchy', '') for r in results if r.get('hierarchy')]
                 if hierarchies:
             print(f"   - 발견된 계층 정보: {len(hierarchies)}개")
     
     return results

# ===== RAG Pipeline =====
def rag_pipeline(query: str, model, client, collection_name: str, top_k: int = 3):
    # 1) Retriever 단계
    results, _ = dense_search(
        query=query,
        model=model,
        client=client,
        collection_name=collection_name,
        top_k=top_k
    )

    # 2) 검색 결과 합치기
    context_text = "\n".join([r["text"] for r in results if r.get("text")])

    # 3) Reader 호출 (Zephyr LLM) - 튜닝된 프롬프트 적용
    prompt = f"""<|system|>
너는 재무보고서 전문가이자 데이터 구조화 전문가다. 
너의 임무는 검색된 문서에서 사용자가 요청한 항목을 
metadata의 account_id, account_name, parent_id, is_total, is_subtotal, period_type,
hierarchy, level 정보를 반드시 참고하여 계층 구조를 반영해 표로 정리하는 것이다.

요구사항:
1. 반드시 metadata의 account_id, account_name, parent_id, is_total, is_subtotal, period_type, hierarchy, level 정보를 모두 활용하라.
2. account_id와 account_name으로 항목을 식별하고, parent_id를 사용하여 상위-하위 관계를 연결하라.
3. is_total과 is_subtotal은 합계/소계 여부를 명확히 표시하라.
4. period_type은 "당기/전기/누적" 등의 기간 구분을 반드시 표에 포함하라.
5. level 값이 커질수록 하위 항목이므로 들여쓰기를 적용하거나, 표에서 level 열을 활용하라.
6. 출력은 반드시 표 형식으로: 
   | account_id | account_name | parent_id | level | hierarchy | 값(백만원) | is_total | is_subtotal | period_type |
7. 모든 항목을 빠짐없이 보여주고, 추측하지 말고 검색된 문서와 metadata만 근거로 작성하라.
8. 만약 유동자산과 관련된 유동부채가 함께 제공된다면, 유동비율(Current Ratio = 유동자산 ÷ 유동부채)을 계산하여 표 맨 아래에 추가하라.
9. 출처는 메타데이터에서 report_year를 사용해서 적어라.
10. 절대로 account_name을 지어서 만들어내지 말라.
</s>
<|user|>
다음은 검색된 문서다:
{context_text}

질문: {query}
</s>
<|assistant|>
"""

    response = llm(prompt, max_tokens=1024, stop=["</s>"])
    return response["choices"][0]["text"].strip()

# ===== 실행 예시 =====
if __name__ == "__main__":
    questions = [
        # "2014년 재무상태표 상 당기 유동자산은 얼마인가?",
        # "2014년 현금흐름표 상 당기 영업활동 현금흐름은 얼마인가?",
        # "2015년 당기 비유동자산은 재무상태표에서 얼마인가?",
        # "2015년 손익계산서 상 당기순이익은 얼마인가?",
        # "2016년 재무상태표 상 당기 단기금융상품은 얼마인가요?",
        # "2016년 포괄손익계산서 상 당기 총포괄이익은 얼마니?",
        # "2016년 자본변동표 상 자기주식의 취득은 얼마인가?",
        # "2017년 당기 매출채권은 재무상태표에 따르면 얼마냐?",
        # "2017년 재무상태표상 전기 현금및현금성자산은 얼마입니까?",
        # "2017년 재무상태표상 당기 매각예정분류자산은 얼마인가요?",
        # "2018년 당기 미수금은 재무상태표에서 얼마인가?",
        # "2018년 손익계산서상 매출총이익은 얼마인가요?",
        # "2019년 재무상태표상 종속기업, 관계기업 및 공동기업 투자는 얼마인가요?",
        # "2019년 현금흐름표 상 이익잉여금 배당은 얼마인가요?",
        # "2019년 손익계산서상 기본주당이익은 얼마인가요?",
        # "2020년 재무상태표 상 자산총계는?",
        # "2020년 손익계산서 상 판매비와관리비는 얼마인가요?",
        # "2021년 재무상태표상 당기 기타포괄손익-공정가치금융자산은 얼마인가요?",
        # "2021년 재무상태표에서 당기 유동비율을 계산하면 얼마인가요?",
        # "2021년 손익계산서 상 당기 금융비용은 얼마인가요?",
        # "2022년 재무상태표상 당기 비유동부채는 얼마인가?",
        # "2022년 손익계산서 상 당기 법인세비용은 얼마니?",
        # "2022년 당기 현금흐름표 상 투자활동 현금흐름은 얼마인가?",
        # "2023년 재무상태표 상 재고자산은 얼마인가?",
        # "2023년 당기 영업이익은 얼마인가?",
        # "2024년에는 재무상태표상 당기 무형자산이 얼마야?",
        # "2024년 재무상태표 상 당기 우선주자본금은 얼마인가?",
        # "2024년 손익계산서상 당기 법인세비용은 얼마야?",
        #"손익계산서 상 매출액이 전년 대비 오른 연도를 전부 알려줘",
        #"유동비율(유동자산/유동부채)을 2014년 재무상태표에서 값을 찾아서 계산해봐"
        "2014년 재무상태표에서 당기 유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해."
    ]

    for q in questions:
        # LLM RAG는 Cell 5에서 실행됩니다
        print("✅ 모든 함수와 Zephyr 모델 로드 완료!")
        print("📝 Cell 5를 실행하여 LLM RAG 파이프라인을 테스트하세요!")
        print("=" * 80)
        print("질문:", q)
        print("답변:", answer)


💬 질문: 2014년 재무상태표에서 당기 유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해.

🔍 검색 결과 (10개):

1. 스코어: 0.6022
   연도: 2014
   텍스트: 재무상태표에서 2014년 (당기) 유동자산는 62,054,773백만원입니다....
   📊 메타데이터: account_id: 자산_유동자산, account_name: 유동자산, parent_id: 자산, level: 2, hierarchy: ['자산', '유동자산'], is_total: False, is_subtotal: False, period_type: current, statement_type: balance
--------------------------------------------------------------------------------

2. 스코어: 0.6022
   연도: 2014
   텍스트: 재무상태표에서 2014년 (당기) 유동자산는 62,054,773백만원입니다....
   📊 메타데이터: account_id: 자산_유동자산, account_name: 유동자산, parent_id: 자산, level: 2, hierarchy: ['자산', '유동자산'], is_total: False, is_subtotal: False, period_type: current, statement_type: balance
--------------------------------------------------------------------------------

3. 스코어: 0.6022
   연도: 2014
   텍스트: 재무상태표에서 2014년 (당기) 유동자산는 62,054,773백만원입니다....
   📊 메타데이터: account_id: 자산_유동자산, account_name: 유동자산, parent_id: 자산, level: 2, hierarchy: ['자산', '유동자산'], is_total: Fals

In [ ]:
# ================== RAG Pipeline with LLM ==================
def rag_pipeline(query: str, model, client, collection_name: str, top_k: int = 20):
    """실제 LLM을 사용하는 RAG 파이프라인"""
    # 1) 검색 단계
    results, _ = dense_search(
        query=query,
        model=model,
        client=client,
        collection_name=collection_name,
        top_k=top_k,
        fallback_min_hits=8
    )
    
    # 2) 컨텍스트 구성 (메타데이터 포함)
    context_parts = []
    for i, r in enumerate(results, 1):
        metadata_info = []
        metadata_keys = ['account_id', 'account_name', 'parent_id', 'level', 'hierarchy', 
                        'is_total', 'is_subtotal', 'period_type', 'statement_type', 'report_year']
        for key in metadata_keys:
            if key in r:
                metadata_info.append(f"{key}: {r[key]}")
        
        context_part = f"문서 {i}:\n"
        context_part += f"텍스트: {r['text']}\n"
        context_part += f"메타데이터: {', '.join(metadata_info)}\n"
        context_parts.append(context_part)
    
    context_text = "\n".join(context_parts)
    
    # 3) LLM 프롬프트 구성
    prompt = f"""<|system|>
너는 재무보고서 전문가이자 데이터 구조화 전문가다.
검색된 문서들의 metadata를 활용하여 사용자 질문에 대한 구조화된 표를 작성해라.

**중요 규칙:**
1. 반드시 metadata의 account_id, account_name, parent_id, level, hierarchy 정보를 모두 활용하라
2. level 값이 클수록 하위 항목이므로 계층 구조를 반영하라
3. 표 형식으로 출력: | account_id | account_name | parent_id | level | hierarchy | 값(백만원) | period_type |
4. 모든 검색된 항목을 빠짐없이 포함하라
5. 추측하지 말고 제공된 데이터만 사용하라
6. hierarchy는 리스트 형태로 표시하라
7. 값은 텍스트에서 추출한 숫자를 사용하라
</s>
<|user|>
질문: {query}

검색된 문서들:
{context_text}

위 정보를 바탕으로 구조화된 표를 작성해주세요.
</s>
<|assistant|>
"""
    
    # 4) LLM 실행
    response = llm(prompt, max_tokens=2048, stop=["</s>"], temperature=0.1)
    return response["choices"][0]["text"].strip()

# ================== LLM 테스트 실행 ==================
# Cell 4에서 검색 함수들이 먼저 실행되었으므로, 이제 LLM RAG를 실행합니다
print("🚀 LLM RAG 파이프라인 테스트 (Cell 4의 검색 결과를 LLM으로 구조화)")
print("=" * 100)

test_question = "2014년 재무상태표에서 당기 비유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해."
print(f"💬 질문: {test_question}")
print("=" * 100)

try:
    # LLM을 사용한 구조화된 답변 생성
    answer = rag_pipeline(test_question, embed_model, client, collection_name, top_k=30)
    
    print("🤖 LLM 답변:")
    print(answer)
    print("\n" + "=" * 100)
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("📝 Cell 4를 먼저 실행했는지 확인하세요!")


In [ ]:
# ================== 🚀 독립 실행형 LLM RAG 파이프라인 ==================
import numpy as np
import json
import re
from pathlib import Path
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
from llama_cpp import Llama

print("🚀 독립 실행형 LLM RAG 파이프라인")
print("=" * 100)

# ===== 설정 =====
QDRANT_PATH = "/Users/dan/Desktop/snu_project/git_제출용/data/vector_store/final-sjchunk/bge-ko-qdrant_db"
MODEL_PATH = "/Users/dan/Desktop/snu_project/models/zephyr-7b-beta.Q4_K_M.gguf"
collection_name = "audit_chunks"

# ===== 모델 로드 =====
print("📦 모델 로드 중...")
embed_model = SentenceTransformer("dragonkue/bge-m3-ko")
client = QdrantClient(path=QDRANT_PATH)
llm = Llama(model_path=MODEL_PATH, n_ctx=4096, n_threads=8, n_gpu_layers=35)
print("   ✅ 모델 로드 완료")

# ===== 유틸 함수 =====
def extract_year_from_query(query: str):
    m = re.search(r"(20\d{2}|19\d{2})", query)
    return int(m.group(1)) if m else None

def detect_statement_type(query: str) -> str:
    q = query.replace(" ", "")
    if "재무상태표" in q or "대차대조표" in q: return "balance"
    if "손익계산서" in q or "포괄손익" in q: return "income"
    if "현금흐름표" in q: return "cashflow"
    if "자본변동표" in q: return "equity_changes"
    return "balance"

def build_filter(year=None, statement_type=None, period_type=None, parent_id=None, min_level=None):
    must = []
    if year: must.append(qmodels.FieldCondition(key="report_year", match=qmodels.MatchValue(value=year)))
    if statement_type: must.append(qmodels.FieldCondition(key="statement_type", match=qmodels.MatchValue(value=statement_type)))
    if period_type: must.append(qmodels.FieldCondition(key="period_type", match=qmodels.MatchValue(value=period_type)))
    if parent_id: must.append(qmodels.FieldCondition(key="parent_id", match=qmodels.MatchValue(value=parent_id)))
    if min_level: must.append(qmodels.FieldCondition(key="level", range=qmodels.Range(gte=min_level)))
    must.append(qmodels.FieldCondition(key="is_total", match=qmodels.MatchValue(value=False)))
    return qmodels.Filter(must=must) if must else None

# ===== 검색 함수 =====
def simple_search(query: str, top_k: int = 20):
    year = extract_year_from_query(query)
    statement_type = detect_statement_type(query)
    period_type = "current"
    
    # 비유동자산 검색을 위한 필터
    query_filter = build_filter(
        year=year,
        statement_type=statement_type,
        period_type=period_type,
        min_level=3  # 하위 항목들
    )
    
    qv = embed_model.encode("query: " + query, normalize_embeddings=True).tolist()
    
    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=query_filter
    )
    
    output = []
    seen_ids = set()
    for r in results.points:
        payload = r.payload or {}
        text = payload.get("text")
        if not text: continue
        
        meta = payload.get("metadata", {})
        if not meta: meta = {k: v for k, v in payload.items() if k != "text"}
        
        account_id = meta.get("account_id")
        if account_id and account_id in seen_ids: continue
        if account_id: seen_ids.add(account_id)
        
        # 비유동자산 관련 필터링
        hierarchy = meta.get("hierarchy", [])
        if isinstance(hierarchy, list) and "비유동자산" in hierarchy:
            item = {"score": r.score, "text": text}
            item.update(meta)
            output.append(item)
    
    output.sort(key=lambda x: (x.get("level", 999), str(x.get("account_name", ""))))
    return output

# ===== RAG 파이프라인 =====
def rag_pipeline(query: str, top_k: int = 20):
    # 1) 검색
    results = simple_search(query, top_k)
    
    # 2) 컨텍스트 구성
    context_parts = []
    for i, r in enumerate(results, 1):
        metadata_keys = ['account_id', 'account_name', 'parent_id', 'level', 'hierarchy', 
                        'is_total', 'is_subtotal', 'period_type', 'statement_type', 'report_year']
        metadata_info = [f"{key}: {r[key]}" for key in metadata_keys if key in r]
        
        context_part = f"문서 {i}:\n텍스트: {r['text']}\n메타데이터: {', '.join(metadata_info)}\n"
        context_parts.append(context_part)
    
    context_text = "\n".join(context_parts)
    
    # 3) LLM 프롬프트
    prompt = f"""<|system|>
너는 재무보고서 전문가다. 검색된 문서들의 metadata를 활용하여 구조화된 표를 작성해라.

**규칙:**
1. metadata의 account_id, account_name, parent_id, level, hierarchy 정보를 모두 활용
2. level 값이 클수록 하위 항목
3. 표 형식: | account_id | account_name | parent_id | level | hierarchy | 값(백만원) | period_type |
4. 모든 검색된 항목을 포함
5. 추측하지 말고 제공된 데이터만 사용
6. 값은 텍스트에서 추출한 숫자 사용
</s>
<|user|>
질문: {query}

검색된 문서들:
{context_text}

위 정보를 바탕으로 구조화된 표를 작성해주세요.
</s>
<|assistant|>
"""
    
    # 4) LLM 실행
    response = llm(prompt, max_tokens=2048, stop=["</s>"], temperature=0.1)
    return response["choices"][0]["text"].strip()

# ===== 실행 =====
test_question = "2014년 재무상태표에서 당기 비유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해."
print(f"💬 질문: {test_question}")
print("=" * 100)

try:
    # 1) 검색
    print("🔍 1단계: 검색 실행 중...")
    search_results = simple_search(test_question, top_k=20)
    print(f"   ✅ {len(search_results)}개 문서 검색 완료")
    
    # 2) LLM 답변 생성
    print("🤖 2단계: LLM 답변 생성 중...")
    answer = rag_pipeline(test_question, top_k=20)
    
    print("🎯 최종 답변:")
    print("=" * 100)
    print(answer)
    print("=" * 100)
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# ================== 🎯 최종 실행 안내 ==================
print("🎯 독립 실행형 RAG 시스템")
print("=" * 50)
print("📝 Cell 6만 실행하시면 모든 기능이 작동합니다!")
print("   - 모델 로드")
print("   - 검색 실행") 
print("   - LLM 답변 생성")
print("=" * 50)
print("⚡ 다른 셀들은 무시하고 Cell 6만 실행하세요!")


In [1]:
import numpy as np
from pathlib import Path
# from llama_cpp import Llama  # 임시로 주석 처리 (모듈이 설치되어 있지 않음)
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
import re
from qdrant_client.http import models as qmodels

# ===== 임베딩 모델 로드 =====
embed_model = SentenceTransformer("dragonkue/bge-m3-ko")

# ===== Qdrant 클라이언트 =====
QDRANT_PATH = "/Users/dan/Desktop/snu_project/git_제출용/data/vector_store/final-sjchunk/bge-ko-qdrant_db"
client = QdrantClient(path=QDRANT_PATH)

collection_name = "audit_chunks"


# =============== 유틸 ===============
def extract_year_from_query(query: str):
    """질문에서 연도(4자리 숫자) 추출"""
    match = re.search(r"(20\d{2}|19\d{2})", query)
    if match:
        return int(match.group(1))
    return None

def extract_numbers(text: str) -> str:
    """텍스트에서 숫자만 추출"""
    if text is None:
        return ""
    return "".join(re.findall(r"\d+", text.replace(",", "")))


# =============== (신규) 필터 빌더 ===============  ### ADDED
def build_filter(
    year: int = None,
    statement_type: str = None,     # e.g., "balance"
    period_type: str = None,        # e.g., "current" or "previous"
    must_have_hierarchy: str = None,# e.g., "유동자산"
    min_level: int = None,          # e.g., 3
    exclude_totals: bool = True,
    exclude_subtotals: bool = False,
):
    """
    Qdrant 서버 측 필터를 조립합니다.
    - hierarchy는 리스트 필드라고 가정하고 '유동자산' 포함 여부를 MatchAny로 체크합니다.
    """
    must = []

    if year is not None:
        must.append(qmodels.FieldCondition(
            key="report_year",
            match=qmodels.MatchValue(value=year)
        ))

    if statement_type:
        must.append(qmodels.FieldCondition(
            key="statement_type",
            match=qmodels.MatchValue(value=statement_type)
        ))

    if period_type:
        must.append(qmodels.FieldCondition(
            key="period_type",
            match=qmodels.MatchValue(value=period_type)
        ))

    if must_have_hierarchy:
        # hierarchy가 ["자산","유동자산","현금및..."] 처럼 리스트라고 가정
        must.append(qmodels.FieldCondition(
            key="hierarchy",
            match=qmodels.MatchAny(any=[must_have_hierarchy])
        ))

    if min_level is not None:
        must.append(qmodels.FieldCondition(
            key="level",
            range=qmodels.Range(gte=min_level)
        ))

    if exclude_totals:
        must.append(qmodels.FieldCondition(
            key="is_total",
            match=qmodels.MatchValue(value=False)
        ))

    if exclude_subtotals:
        must.append(qmodels.FieldCondition(
            key="is_subtotal",
            match=qmodels.MatchValue(value=False)
        ))

    return qmodels.Filter(must=must) if must else None


# =============== dense_search ===============  ### CHANGED (핵심 수정)
def dense_search(query: str, model, client, collection_name: str,
                 top_k: int = 50, ground_truth=None,
                 score_threshold: float = None,
                 strict_children_of: str = None):
    """
    Qdrant query_points 기반 Dense Retriever + 성능지표 계산
    변경 사항:
    - BGE 계열 권장: 쿼리에 "query: " 프리픽스 적용
    - 서버 필터 강화: balance/current/연도/유동자산 포함/level>=3/토탈 제외
    - 결과 중복 제거(account_id)
    - strict_children_of(예: '유동자산') 재검증
    - 결과 정렬(level, account_name)
    - 필요 시 score 하한선 적용
    """
    # 0) 연도 추출
    year = extract_year_from_query(query)

    # 1) 쿼리 임베딩 (BGE는 질의 프리픽스가 미세하게 도움됨)
    qv = model.encode("query: " + query, normalize_embeddings=True).tolist()  # ### CHANGED

    # 2) 서버 필터 구성
    query_filter = build_filter(
        year=year,
        statement_type="balance",           # 재무상태표 강제
        period_type="current",              # 당기값 강제
        must_have_hierarchy=strict_children_of or "유동자산",
        min_level=3,                        # 하위계층만
        exclude_totals=True,                # 합계 제외
        exclude_subtotals=False             # 필요 시 True로
    )

    # 3) 검색 실행
    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=query_filter
    )

    # 4) 결과 정리 + 중복 제거(account_id)
    seen_ids = set()
    output = []
    for r in results.points:
        payload = r.payload or {}
        text = payload.get("text")
        if not text:
            continue

        # 메타 평탄화
        meta = {}
        if "metadata" in payload and isinstance(payload["metadata"], dict):
            meta.update(payload["metadata"])
        else:
            meta.update({k: v for k, v in payload.items() if k != "text"})

        # account_id 기준 중복 제거
        account_id = meta.get("account_id")
        if account_id and account_id in seen_ids:
            continue
        if account_id:
            seen_ids.add(account_id)

        # 스코어 컷
        if score_threshold is not None and r.score < score_threshold:
            continue

        # 엄격 재검증: hierarchy에 특정 노드가 반드시 포함되어야 함
        if strict_children_of:
            hier = meta.get("hierarchy", [])
            if isinstance(hier, list) and strict_children_of not in hier:
                continue

        result_item = {"score": r.score, "text": text}
        result_item.update(meta)
        output.append(result_item)

    # 5) 가독 정렬: level 오름차순 → account_name
    output.sort(key=lambda x: (x.get("level", 999), str(x.get("account_name", ""))))

    # 6) 성능 지표 (옵션)
    metrics = {}
    if ground_truth:
        normalized_gt = [extract_numbers(gt) for gt in ground_truth]
        used = set()
        relevances = []
        for r in output[:top_k]:
            nums = extract_numbers(r["text"])
            hit = 0
            for gt in normalized_gt:
                if gt and gt in nums and gt not in used:
                    hit = 1
                    used.add(gt)
                    break
            relevances.append(hit)

        precision = sum(relevances) / top_k if top_k > 0 else 0.0
        recall = min(sum(relevances), len(normalized_gt)) / len(normalized_gt) if normalized_gt else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
        rr = next((1.0 / (i+1) for i, rel in enumerate(relevances) if rel == 1), 0.0)
        dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevances))
        ideal_hits = min(len(normalized_gt), top_k)
        idcg = sum(1.0 / np.log2(i + 2) for i in range(ideal_hits))
        ndcg = dcg / idcg if idcg > 0 else 0.0

        metrics = {"Precision@3": precision, "Recall@3": recall, "F1@3": f1, "MRR": rr, "nDCG@3": ndcg}

    return output, metrics


# =============== 간단 검색 출력 ===============  ### CHANGED (새 파라미터 반영)
def simple_search_test(query: str, model, client, collection_name: str, top_k: int = 10,
                       score_threshold: float = None, strict_children_of: str = None):
    """LLM 없이 검색 결과만 확인하는 함수"""
    results, _ = dense_search(
        query=query,
        model=model,
        client=client,
        collection_name=collection_name,
        top_k=top_k,
        score_threshold=score_threshold,        # ### CHANGED
        strict_children_of=strict_children_of   # ### CHANGED
    )

    print(f"🔍 검색 결과 ({len(results)}개):")
    print("=" * 100)

    for i, r in enumerate(results, 1):
        print(f"\n{i}. 스코어: {r['score']:.4f}")
        print(f"   연도: {r.get('report_year', 'N/A')}")
        print(f"   텍스트: {r['text'][:200]}...")

        metadata_keys = ['account_id', 'account_name', 'parent_id', 'level', 'hierarchy',
                         'is_total', 'is_subtotal', 'period_type', 'statement_type']
        metadata_info = []
        for key in metadata_keys:
            if key in r:
                metadata_info.append(f"{key}: {r[key]}")
        if metadata_info:
            print(f"   📊 메타데이터: {', '.join(metadata_info)}")
        print("-" * 80)

    return results


# =============== RAG Pipeline (LLM 없이 검색만) ===============  ### CHANGED
def rag_pipeline_simple(query: str, model, client, collection_name: str, top_k: int = 10,
                        score_threshold: float = None, strict_children_of: str = None):
    """LLM 없이 검색 결과만 반환하는 간단한 RAG"""
    print(f"💬 질문: {query}\n")
    results = simple_search_test(
        query, model, client, collection_name, top_k=top_k,
        score_threshold=score_threshold, strict_children_of=strict_children_of
    )
    print(f"\n📋 요약:")
    print(f"   - 총 {len(results)}개의 관련 문서를 찾았습니다.")
    if results:
        years = list({r.get('report_year') for r in results if r.get('report_year')})
        if years:
            print(f"   - 관련 연도: {', '.join(map(str, sorted(years)))}")
        hierarchies = [r.get('hierarchy', []) for r in results if r.get('hierarchy')]
        if hierarchies:
            print(f"   - 발견된 계층 정보: {len(hierarchies)}개")
    return results


# ===== Zephyr 모델 로드 (임시로 주석 처리) =====
# model_path = Path("/Users/bag-yebin/Desktop/흠/자연어처리/samsun-audit-rag-qa/models/zephyr-7b-beta.Q4_K_M.gguf").resolve()
# llm = Llama(
#     model_path=str(model_path),
#     n_ctx=4096,
#     n_threads=8,
#     n_gpu_layers=35
# )


# =============== 실행 예시 ===============
if __name__ == "__main__":
    questions = [
        "2014년 재무상태표에서 당기 유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해."
    ]

    for q in questions:
        print("=" * 100)
        # strict_children_of="유동자산" 을 안전망으로 활용
        results = rag_pipeline_simple(
            q, embed_model, client, collection_name,
            top_k=50,
            score_threshold=0.55,           # 필요 시 조절
            strict_children_of="유동자산"   # '유동자산' subtree만
        )
        print("\n" + "=" * 100)


💬 질문: 2014년 재무상태표에서 당기 유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해.

🔍 검색 결과 (1개):

1. 스코어: 0.5557
   연도: 2014
   텍스트: 재무상태표에서 2014년 (당기) 기타유동자산는 821,079백만원입니다....
   📊 메타데이터: account_id: 자산_유동자산_기타유동자산, account_name: 기타유동자산, parent_id: 자산_유동자산, level: 3, hierarchy: ['유동자산', '기타유동자산'], is_total: False, is_subtotal: True, period_type: current, statement_type: balance
--------------------------------------------------------------------------------

📋 요약:
   - 총 1개의 관련 문서를 찾았습니다.
   - 관련 연도: 2014
   - 발견된 계층 정보: 1개



In [1]:
import numpy as np
from pathlib import Path
# from llama_cpp import Llama  # 임시로 주석 처리 (모듈이 설치되어 있지 않음)
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
import re
from qdrant_client.http import models as qmodels

# ===== 임베딩 모델 로드 =====
embed_model = SentenceTransformer("dragonkue/bge-m3-ko")

# ===== Qdrant 클라이언트 =====
QDRANT_PATH = "/Users/dan/Desktop/snu_project/git_제출용/data/vector_store/final-sjchunk/bge-ko-qdrant_db"
client = QdrantClient(path=QDRANT_PATH)

collection_name = "audit_chunks"

# ================== 유틸 ==================
KOREAN_SPACE_RE = re.compile(r"(?:[가-힣]\s)+(?:[가-힣])")
def collapse_ko_spaced(s: str) -> str:
    """'유  동  자  산' -> '유동자산'"""
    if not s:
        return s
    return s.replace(" ", "") if KOREAN_SPACE_RE.fullmatch(s) else s

def extract_year_from_query(query: str):
    m = re.search(r"(20\d{2}|19\d{2})", query)
    return int(m.group(1)) if m else None

def extract_numbers(text: str) -> str:
    if text is None: return ""
    return "".join(re.findall(r"\d+", text.replace(",", "")))

def detect_statement_type(query: str) -> str:
    q = query.replace(" ", "")
    if "재무상태표" in q or "대차대조표" in q: return "balance"
    if "손익계산서" in q or "포괄손익" in q: return "income"
    if "현금흐름표" in q: return "cashflow"
    if "자본변동표" in q: return "equity_changes"
    # 기본값(질문에 없으면 재무상태표로 가정)
    return "balance"

# ================== Qdrant Filter 빌더 ==================
def build_filter(
    year: int = None,
    statement_type: str = None,
    period_type: str = None,
    must_have_hierarchy: str = None,  # 리스트 포함 체크
    parent_id: str = None,
    min_level: int = None,
    exclude_totals: bool = True,
    exclude_subtotals: bool = False,
):
    must = []
    if year is not None:
        must.append(qmodels.FieldCondition(key="report_year", match=qmodels.MatchValue(value=year)))
    if statement_type:
        must.append(qmodels.FieldCondition(key="statement_type", match=qmodels.MatchValue(value=statement_type)))
    if period_type:
        must.append(qmodels.FieldCondition(key="period_type", match=qmodels.MatchValue(value=period_type)))
    if must_have_hierarchy:
        must.append(qmodels.FieldCondition(key="hierarchy", match=qmodels.MatchAny(any=[must_have_hierarchy])))
    if parent_id:
        must.append(qmodels.FieldCondition(key="parent_id", match=qmodels.MatchValue(value=parent_id)))
    if min_level is not None:
        must.append(qmodels.FieldCondition(key="level", range=qmodels.Range(gte=min_level)))
    if exclude_totals:
        must.append(qmodels.FieldCondition(key="is_total", match=qmodels.MatchValue(value=False)))
    if exclude_subtotals:
        must.append(qmodels.FieldCondition(key="is_subtotal", match=qmodels.MatchValue(value=False)))
    return qmodels.Filter(must=must) if must else None

# ================== Parent 자동 탐색 ==================
def resolve_parent_node(
    query: str,
    client: QdrantClient,
    collection_name: str,
    year: int,
    statement_type: str = "balance",
    period_type: str = "current",
    level: int = 2,
    top_k: int = 10,
):
    """
    질의에서 '부모 노드'(예: 유동자산, 비유동자산, 유동부채 등)를 자동으로 찾아 반환.
    - 방법: 쿼리 임베딩 + 서버 필터(year/statement/current/level=2)로 상위 후보를 받고,
            후보의 account_name/hierarchy 말단이 질의 문자열에 등장하는지 우선 매칭.
    - 반환: dict(account_id, account_name, hierarchy, level, report_year, ...)
    """
    # 1) 필터: 해당 연도, 표 종류, 당기, level=2 (부모 레벨)
    filt = build_filter(
        year=year,
        statement_type=statement_type,
        period_type=period_type,
        min_level=level,
        exclude_totals=True,
        exclude_subtotals=False,
    )
    # 2) 질의 임베딩
    qv = embed_model.encode("query: " + query, normalize_embeddings=True).tolist()
    # 3) 검색
    res = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=filt
    )

    # 4) 후보 중 '질문에 이름이 직접 등장'하는 것을 우선 선택
    q_norm = collapse_ko_spaced(query.replace(" ", ""))
    best = None
    for p in res.points:
        payload = p.payload or {}
        meta = payload.get("metadata", payload)
        name = collapse_ko_spaced(str(meta.get("account_name", "")))
        # hierarchy의 말단 노드명도 검사
        last_h = ""
        hier = meta.get("hierarchy", [])
        if isinstance(hier, list) and len(hier) > 0:
            last_h = collapse_ko_spaced(str(hier[-1]))
        # 직접 문자열 매칭
        hit = False
        if name and name in q_norm:
            hit = True
        elif last_h and last_h in q_norm:
            hit = True
        # '유동 자산'처럼 띄어쓰기 포함 질의를 대비해 축약 매칭도 검사 (이미 q_norm에서 공백 제거)
        if hit:
            best = meta
            break

    # 5) 매칭이 없다면 스코어 1순위로
    if not best and res.points:
        best_payload = res.points[0].payload or {}
        best = best_payload.get("metadata", best_payload)

    return best  # 실패 시 None

# ================== 자식 전량 회수(scroll) ==================
def scroll_children_by_parent(
    client: QdrantClient,
    collection_name: str,
    year: int,
    parent_id: str,
    statement_type: str = "balance",
    period_type: str = "current",
    min_level: int = 3,
    exclude_totals: bool = True,
    exclude_subtotals: bool = False,
    limit: int = 256
):
    filt = build_filter(
        year=year,
        statement_type=statement_type,
        period_type=period_type,
        parent_id=parent_id,
        min_level=min_level,
        exclude_totals=exclude_totals,
        exclude_subtotals=exclude_subtotals
    )
    out = []
    next_offset = None
    while True:
        points, next_offset = client.scroll(
            collection_name=collection_name,
            scroll_filter=filt,
            with_payload=True,
            with_vectors=False,
            limit=limit,
            offset=next_offset
        )
        for p in points:
            payload = p.payload or {}
            text = payload.get("text")
            if not text:
                continue
            meta = payload.get("metadata", {})
            if not meta:
                meta = {k: v for k, v in payload.items() if k != "text"}
            item = {"score": None, "text": text}
            item.update(meta)
            out.append(item)
        if not next_offset:
            break
    out.sort(key=lambda x: (x.get("level", 999), str(x.get("account_name", ""))))
    return out

# ================== Dense Search (자동 parent 사용) ==================
def dense_search(query: str, model, client, collection_name: str,
                 top_k: int = 50, ground_truth=None,
                 score_threshold: float = None,
                 strict_children_of: str = None,
                 fallback_min_hits: int = 6):
    """
    1) 질의로부터 연도/표종류/부모노드 자동 추정
    2) 부모 account_id를 parent_id_hint로 사용하여 자식 검색
    3) 부족하면 scroll로 전량 보강
    """
    # 0) 연도/표종류 추론
    year = extract_year_from_query(query)
    statement_type = detect_statement_type(query)
    period_type = "current"  # 질문이 '당기' 중심이므로 기본값 current

    # 연도는 반드시 필요. 없다면 필터 약화(=전 연도) 대신 결과 품질 위해 None 허용 but fallback에서 year 필요
    # → year 없으면 우선 dense만 수행하고, fallback은 생략하거나 가장 최근 연도 찾기 로직 추가 가능
    # 여기서는 year 없으면 fallback 생략
    # 1) 부모 노드 자동 탐색
    parent_meta = None
    if year is not None:
        parent_meta = resolve_parent_node(
            query=query,
            client=client,
            collection_name=collection_name,
            year=year,
            statement_type=statement_type,
            period_type=period_type,
            level=2,
            top_k=10
        )

    # 2) 쿼리 임베딩
    qv = model.encode("query: " + query, normalize_embeddings=True).tolist()

    # 3) 서버 필터 구성 (부모를 찾았으면 parent_id 강제)
    query_filter = build_filter(
        year=year,
        statement_type=statement_type,
        period_type=period_type,
        parent_id=parent_meta.get("account_id") if parent_meta else None,
        min_level=3,
        exclude_totals=True,
        exclude_subtotals=False
    )

    # 4) 검색 실행
    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=query_filter
    )

    # 5) 결과 정리 + 중복 제거
    seen_ids = set()
    output = []
    for r in results.points:
        payload = r.payload or {}
        text = payload.get("text")
        if not text:
            continue
        meta = payload.get("metadata", {})
        if not meta:
            meta = {k: v for k, v in payload.items() if k != "text"}

        account_id = meta.get("account_id")
        if account_id and account_id in seen_ids:
            continue
        if account_id:
            seen_ids.add(account_id)

        if score_threshold is not None and r.score < score_threshold:
            continue

        if strict_children_of:
            hier = meta.get("hierarchy", [])
            if isinstance(hier, list) and strict_children_of not in hier:
                continue

        item = {"score": r.score, "text": text}
        item.update(meta)
        output.append(item)

    # 6) 가독 정렬
    output.sort(key=lambda x: (x.get("level", 999), str(x.get("account_name", ""))))

    # 7) Fallback: 결과가 부족할 때 parent_id로 전체 회수
    if year is not None and parent_meta and len(output) < fallback_min_hits:
        fallback_items = scroll_children_by_parent(
            client=client,
            collection_name=collection_name,
            year=year,
            parent_id=parent_meta["account_id"],
            statement_type=statement_type,
            period_type=period_type,
            min_level=3,
            exclude_totals=True,
            exclude_subtotals=False
        )
        # 병합(중복 제거)
        by_id = {x.get("account_id"): x for x in output if x.get("account_id")}
        for it in fallback_items:
            aid = it.get("account_id")
            if aid and aid not in by_id:
                by_id[aid] = it
        output = list(by_id.values())
        output.sort(key=lambda x: (x.get("level", 999), str(x.get("account_name", ""))))

    # 8) 메트릭(옵션)
    metrics = {}
    if ground_truth:
        normalized_gt = [extract_numbers(gt) for gt in ground_truth]
        used = set()
        relevances = []
        for r in output[:top_k]:
            nums = extract_numbers(r["text"])
            hit = 0
            for gt in normalized_gt:
                if gt and gt in nums and gt not in used:
                    hit = 1
                    used.add(gt)
                    break
            relevances.append(hit)

        precision = sum(relevances) / top_k if top_k > 0 else 0.0
        recall = min(sum(relevances), len(normalized_gt)) / len(normalized_gt) if normalized_gt else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
        rr = next((1.0 / (i+1) for i, rel in enumerate(relevances) if rel == 1), 0.0)
        dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevances))
        ideal_hits = min(len(normalized_gt), top_k)
        idcg = sum(1.0 / np.log2(i + 2) for i in range(ideal_hits))
        ndcg = dcg / idcg if idcg > 0 else 0.0
        metrics = {"Precision@3": precision, "Recall@3": recall, "F1@3": f1, "MRR": rr, "nDCG@3": ndcg}

    return output, metrics

# ================== 출력 헬퍼 ==================
def simple_search_test(query: str, model, client, collection_name: str, top_k: int = 10,
                       score_threshold: float = None, strict_children_of: str = None,
                       fallback_min_hits: int = 6):
    results, _ = dense_search(
        query=query,
        model=model,
        client=client,
        collection_name=collection_name,
        top_k=top_k,
        score_threshold=score_threshold,
        strict_children_of=strict_children_of,
        fallback_min_hits=fallback_min_hits
    )
    print(f"🔍 검색 결과 ({len(results)}개):")
    print("=" * 100)
    for i, r in enumerate(results, 1):
        print(f"\n{i}. 스코어: {r.get('score', 0) if r.get('score') is not None else float('nan'):.4f}")
        print(f"   연도: {r.get('report_year', 'N/A')}")
        print(f"   텍스트: {r['text'][:200]}...")
        metadata_keys = ['account_id', 'account_name', 'parent_id', 'level', 'hierarchy',
                         'is_total', 'is_subtotal', 'period_type', 'statement_type']
        metadata_info = []
        for key in metadata_keys:
            if key in r:
                metadata_info.append(f"{key}: {r[key]}")
        if metadata_info:
            print(f"   📊 메타데이터: {', '.join(metadata_info)}")
        print("-" * 80)
    return results

def rag_pipeline_simple(query: str, model, client, collection_name: str, top_k: int = 10,
                        score_threshold: float = None, strict_children_of: str = None,
                        fallback_min_hits: int = 6):
    print(f"💬 질문: {query}\n")
    results = simple_search_test(
        query, model, client, collection_name, top_k=top_k,
        score_threshold=score_threshold, strict_children_of=strict_children_of,
        fallback_min_hits=fallback_min_hits
    )
    print(f"\n📋 요약:")
    print(f"   - 총 {len(results)}개의 관련 문서를 찾았습니다.")
    if results:
        years = list({r.get('report_year') for r in results if r.get('report_year')})
        if years:
            print(f"   - 관련 연도: {', '.join(map(str, sorted(years)))}")
        hierarchies = [r.get('hierarchy', []) for r in results if r.get('hierarchy')]
        if hierarchies:
            print(f"   - 발견된 계층 정보: {len(hierarchies)}개")
    return results

# ===== Zephyr 모델 로드 =====
model_path = Path("/Users/dan/Desktop/snu_project/models/zephyr-7b-beta.Q4_K_M.gguf").resolve()

from llama_cpp import Llama
llm = Llama(
    model_path=str(model_path),
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=35  # Apple Silicon의 경우 Metal GPU 사용
)

# ================== 실행 예시 ==================
if __name__ == "__main__":
    questions = [
        "2014년 재무상태표에서 당기 비유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해."
    ]
    for q in questions:
        print("=" * 100)
        results = rag_pipeline_simple(
            q, embed_model, client, collection_name,
            top_k=50,
            score_threshold=0.0,      # 컷 없이 다 모으고 부족하면 fallback
            strict_children_of=None,  # parent 자동 해석을 쓰므로 굳이 필요 없음
            fallback_min_hits=8
        )
        print("\n" + "=" * 100)



llama_model_load_from_file_impl: using device Metal (Apple M2 Pro) - 7888 MiB free
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/dan/Desktop/snu_project/models/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:         

💬 질문: 2014년 재무상태표에서 당기 비유동자산의 하위계층 정보를 전부 줘. metadata에서 hierarchy / level을 꼭 참고해.

🔍 검색 결과 (7개):

1. 스코어: 0.5709
   연도: 2014
   텍스트: 재무상태표에서 2014년 (당기) 기타비유동자산는 1,694,436백만원입니다....
   📊 메타데이터: account_id: 자산_비유동자산_기타비유동자산, account_name: 기타비유동자산, parent_id: 자산_비유동자산, level: 3, hierarchy: ['비유동자산', '기타비유동자산'], is_total: False, is_subtotal: True, period_type: current, statement_type: balance
--------------------------------------------------------------------------------

2. 스코어: 0.5290
   연도: 2014
   텍스트: 재무상태표에서 2014년 (당기) 무형자산는 3,051,564백만원입니다. 주석: 14...
   📊 메타데이터: account_id: 자산_비유동자산_무형자산, account_name: 무형자산, parent_id: 자산_비유동자산, level: 3, hierarchy: ['비유동자산', '무형자산'], is_total: False, is_subtotal: True, period_type: current, statement_type: balance
--------------------------------------------------------------------------------

3. 스코어: 0.4806
   연도: 2014
   텍스트: 재무상태표에서 2014년 (당기) 순확정급여자산는 135,951백만원입니다. 주석: 17...
   📊 메타데이터: account_id: 자산_비유동자산_순확정급여자산, account_name: 순확정급여자산, pa